## First, let's load the test set

In [8]:
%load_ext autoreload
%autoreload 2

from averitec import Datapoint
from evidence_generation import EvidenceGenerationResult, GptEvidenceGenerator
from classification import DefaultClassifier
from retrieval import RetrievalResult, SimpleFaissRetriever
from pipeline import Pipeline
import json, random, pickle
from tqdm.notebook import tqdm

random.seed(111)
batch = []
split = "test"
path = "/mnt/data/factcheck/averitec-data/"
with open(path + f"data/{split}.json") as f:
    dataset = json.load(f)
    datapoints = [Datapoint.from_dict(d) for d in dataset]
    
    
class GptBatchedEvidenceGenerator(GptEvidenceGenerator):
    def get_batch_dict(self, datapoint: Datapoint, retrieval_result: RetrievalResult):
        system_prompt = self.format_system_prompt(retrieval_result)
        user_prompt = datapoint.claim
        return {
            "custom_id": f"{split}-{datapoint.claim_id}", 
            "method": "POST", "url": "/v1/chat/completions", 
            "body": {
                "model": "gpt-4o",
                #"model": "gpt-3.5-turbo-0125",
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
            "temperature": 0}}
    
    def __call__(self, datapoint: Datapoint, retrieval_result: RetrievalResult, *args, **kwargs) -> EvidenceGenerationResult:
        batch.append(self.get_batch_dict(datapoint, retrieval_result))
        return EvidenceGenerationResult(evidences=[],metadata={"suggested_label":[0,0,0,0]})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
pipeline = Pipeline(
    SimpleFaissRetriever(path + "data_store/vecstore/test/2k"), # TODO: TRIPLE CHECK BEFORE EVERY LAUNCH
    GptBatchedEvidenceGenerator(), 
    DefaultClassifier()
)

In [21]:
len(datapoints)

2215

In [22]:
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/submission_{split}.json", "w") as f:
        json.dump(submission, f, indent=4)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_{split}.pkl", "wb") as f:
        pickle.dump(dump, f)

  0%|          | 0/2215 [00:00<?, ?it/s]

In [ ]:
/mnt/data/factcheck/averitec-data